<a href="https://colab.research.google.com/github/tiara070403/skripsi/blob/main/sripsifiksbanget.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Sastrawi
!pip install tweet-preprocessor
!pip install textblob
!pip install wordcloud
!pip install nltk

import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from wordcloud import WordCloud, STOPWORDS
import nltk
nltk.download('punkt')
import Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from transformers import pipeline


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.1 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
df = pd.read_csv('/content/crawling.csv', index_col=0)

In [3]:
df_copy = df.copy()

In [4]:
df = df[['created_at', 'full_text']]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5349 entries, 1858070162362204643 to 1230765438897815552
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   created_at  5349 non-null   object
 1   full_text   5349 non-null   object
dtypes: object(2)
memory usage: 125.4+ KB


In [6]:
df['full_text'] = df['full_text'].astype(str).str.lower()  # Mengubah teks menjadi lowercase
df['full_text'] = df['full_text'].replace('false', pd.NA)
df = df.dropna(subset=['full_text'])

In [7]:
df = df.drop_duplicates(subset=['full_text'])

In [8]:
df.duplicated().sum()

np.int64(0)

In [20]:
print(f"\nJumlah duplikat setelah drop: {df.duplicated().sum()}")


Jumlah duplikat setelah drop: 10


In [9]:
df = df.dropna()

In [21]:
print(f"\nJumlah nilai null setelah dropna: \n{df.isnull().sum()}")


Jumlah nilai null setelah dropna: 
created_at    0
full_text     0
dtype: int64


In [11]:
df.isnull().sum()

,0
created_at,0
full_text,0


In [22]:
# Case Folding
df['full_text'] = df['full_text'].str.lower()
print("\nDataFrame setelah Case Folding:")
print(df.head())


DataFrame setelah Case Folding:
                                         created_at  \
conversation_id_str                                   
1858070162362204643  Sun Nov 17 08:50:22 +0000 2024   
1846529554657239219  Wed Oct 16 12:32:07 +0000 2024   
1467691301155979267  Wed Oct 09 13:58:44 +0000 2024   
1836014724523458593  Tue Sep 17 12:09:56 +0000 2024   
1835164951125917967  Sun Sep 15 03:53:14 +0000 2024   

                                                             full_text  
conversation_id_str                                                     
1858070162362204643                lucu banget mahasiswa pencinta alam  
1846529554657239219  rusak mahasiswa pencinta alam jambi menyerang ...  
1467691301155979267  perkumpulan pencinta alam lahir tahun 1953 did...  
1836014724523458593  siaran pers menghadap ukamu ukamu unit pandu l...  
1835164951125917967  siaran pers menghadap ukamu ukamu unit pandu l...  


/tmp/ipython-input-22-2702663551.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_text'] = df['full_text'].str.lower()


In [23]:
# Cleaning
def clean_twitter_text(text):
  text = re.sub(r'@[A-Za-z0-9_]+', '', text)
  text = re.sub(r'#\w+', '', text)
  text = re.sub(r'RT[\s]+', '', text)
  text = re.sub(r'https?://\S+', '', text)
  text = re.sub(r'[^A-Za-z0-9 ]', '', text)
  text = re.sub(r'\s+', ' ', text).strip()
  return text
df['full_text'] = df['full_text'].apply(clean_twitter_text)
print(f"\nShape setelah Cleaning: {df.shape}")
print("DataFrame setelah Cleaning:")
print(df.head())


Shape setelah Cleaning: (5170, 2)
DataFrame setelah Cleaning:
                                         created_at  \
conversation_id_str                                   
1858070162362204643  Sun Nov 17 08:50:22 +0000 2024   
1846529554657239219  Wed Oct 16 12:32:07 +0000 2024   
1467691301155979267  Wed Oct 09 13:58:44 +0000 2024   
1836014724523458593  Tue Sep 17 12:09:56 +0000 2024   
1835164951125917967  Sun Sep 15 03:53:14 +0000 2024   

                                                             full_text  
conversation_id_str                                                     
1858070162362204643                lucu banget mahasiswa pencinta alam  
1846529554657239219  rusak mahasiswa pencinta alam jambi menyerang ...  
1467691301155979267  perkumpulan pencinta alam lahir tahun 1953 did...  
1836014724523458593  siaran pers menghadap ukamu ukamu unit pandu l...  
1835164951125917967  siaran pers menghadap ukamu ukamu unit pandu l...  


/tmp/ipython-input-23-2312499976.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_text'] = df['full_text'].apply(clean_twitter_text)


In [25]:
def filter_tokens_by_length(df, column, min_words, max_words):
    words_count = df[column].astype(str).apply(lambda x: len(x.split()))
    mask = (words_count >= min_words) & (words_count <= max_words)
    filtered_df = df[mask]
    return filtered_df
min_words = 3
max_words = 4000
df = filter_tokens_by_length(df, 'full_text', min_words, max_words)
print(f"\nShape setelah Filter Token Length: {df.shape}")
print("DataFrame setelah Filter Token Length:")
print(df.head())


Shape setelah Filter Token Length: (5151, 2)
DataFrame setelah Filter Token Length:
                                         created_at  \
conversation_id_str                                   
1858070162362204643  Sun Nov 17 08:50:22 +0000 2024   
1846529554657239219  Wed Oct 16 12:32:07 +0000 2024   
1467691301155979267  Wed Oct 09 13:58:44 +0000 2024   
1836014724523458593  Tue Sep 17 12:09:56 +0000 2024   
1835164951125917967  Sun Sep 15 03:53:14 +0000 2024   

                                                             full_text  
conversation_id_str                                                     
1858070162362204643                lucu banget mahasiswa pencinta alam  
1846529554657239219  rusak mahasiswa pencinta alam jambi menyerang ...  
1467691301155979267  perkumpulan pencinta alam lahir tahun 1953 did...  
1836014724523458593  siaran pers menghadap ukamu ukamu unit pandu l...  
1835164951125917967  siaran pers menghadap ukamu ukamu unit pandu l...  


In [26]:
# Normalisasi Data
norm = {'kekekekegiatanananan':'kegiatan', 'press release': 'siaran pers', 'ukamu': 'unit kegiatan mahasiswa', 'giat': 'kegiatan', 'luring': 'luring', 'sekre': 'sekretariat', 'mapala': 'mahasiswa pencinta alam', 'kocak': 'lucu', 'enak': 'enak', 'bilang': 'bilang', 'sowan': 'mengunjungi', 'buka': 'membuka', 'sampai': 'hingga', 'pukul': 'jam', 'sd': 'sampai', 'kapan': 'kapan', 'mau': 'ingin', 'join': 'bergabung', 'sama': 'bersama', 'bisa': 'dapat', 'bantu': 'membantu', 'kamu': 'kamu', 'sama': 'bersama', 'bisa': 'dapat', 'sama': 'bersama', ' mikir2 ':' mikir ', 'perkoempoelan':'perkumpulan', 'sowan':'menghadap', 'genjot':'menyerang', 'wkwkw':'hahaha', 'gtgtgt':'.', 'th':'tahun', 'ukamu':'ukm', 'bejat':'rusak', 'pentjinta':'pencinta', 'pecinta':'pencinta', ' siaaapp ':' siap ', 'okaaay ':'oke ', 'udh ':'sudah ','ga ':'tidak ',' gaskeun':' ayo ', 'wowww ':'wow ', ' haaayyuukkk ':' ayo ', ' yg ':' yang ', ' udh ':' udah ', 'wkwk ':' ', ' min ':' kak ', ' malem ':' malam', ' malem2 ':' malam ', ' sm ':' sama ', ' dy ':' dia ', ' lg ':' lagi ', ' skrg ':' sekarang ', ' ddpn ':' didepan ', ' makasi ':' makasih ', ' pertamaz ':' pertamax ', ' jg ':' juga ', ' donk ':' dong ', ' ikutann ':' ikutan ', ' banyakk ':' banyak ', ' twt ':' tweet', 'mantaap ':'mantap ', ' juarak':' juara ', 'daridulu ':'dari dulu ', 'siapp ':'siap ', ' gamau ':' tidak mau ', ' sll ':' selalu ', ' qu ':' aku ', ' krn ':' karena ', ' irii':' iri', ' muluu ':' terus ', 'mada ':'masa ', 'jgn ':'jangan ', ' jgn ':' jangan ', ' muluuu ':' terus ', 'ntar ':'nanti ', ' awtnya':' awetnya', 'gg ':'keren ', ' kerennn':' keren ', ' bisaa ':' bisa ', 'gaaa':'tidak ', " yg ": " yang ", ' nyampe':' sampai', ' nyampe ':' sampai ', ' lu ':' kamu ', ' ikhlaaasss ':' ikhlas ', ' gak ':' tidak ', ' klo ':' kalo ', ' amp ': ' sampai ', ' ga ':' tidak ', ' yaaaa':' ya ', 'betolll ':'betul ', ' kaga ':' tidak ', ' idk ':' tidak tahu ', ' jkt ':' jakarta ', ' lo ':' kamu ', ' bjir ':' ', ' kek ':' seperti ', ' yg ':' yang ', ' utk ':' untuk ', 'kismin ':'miskin ', ' kismin ':' miskin ', ' pd ':' pada ', ' dgn ':' dengan ', ' ituu ':' itu ', ' jg ':' juga ', 'yoi':'iya ', ' yoi ':' iya ', 'org2 ':'orang ', ' tak ':' tidak ', ' kyk ':' seperti ', ' sbg ':' sebagai ', ' anjjjj ':' ', ' bgt ':' banget ', 'km ':'kamu ', ' km ':' kamu', ' byk ':' banyak ', ' lg ':' lagi ', ' mrk ':' mereka ', ' blm ':' belum '}
def normalisasi(str_text):
  for i in norm:
    str_text = str_text.replace(i, norm[i])
  return str_text
df['full_text'] = df['full_text'].apply(lambda x: normalisasi(x))
print("\nDataFrame setelah Normalisasi:")
print(df.head())


DataFrame setelah Normalisasi:
                                         created_at  \
conversation_id_str                                   
1858070162362204643  Sun Nov 17 08:50:22 +0000 2024   
1846529554657239219  Wed Oct 16 12:32:07 +0000 2024   
1467691301155979267  Wed Oct 09 13:58:44 +0000 2024   
1836014724523458593  Tue Sep 17 12:09:56 +0000 2024   
1835164951125917967  Sun Sep 15 03:53:14 +0000 2024   

                                                             full_text  
conversation_id_str                                                     
1858070162362204643                lucu banget mahasiswa pencinta alam  
1846529554657239219  rusak mahasiswa pencinta alam jambi menyerang ...  
1467691301155979267  perkumpulan pencinta alam lahir tahun 1953 did...  
1836014724523458593  siaran pers menghadap ukamu ukamu unit pandu l...  
1835164951125917967  siaran pers menghadap ukamu ukamu unit pandu l...  


/tmp/ipython-input-26-2327349263.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_text'] = df['full_text'].apply(lambda x: normalisasi(x))


In [27]:
# Stopwords Removal
import Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
more_stop_words = ['anjayyy']
stop_words = StopWordRemoverFactory().get_stop_words()
stop_words.extend(more_stop_words)
new_array = ArrayDictionary(stop_words)
stop_words_remover_new = StopWordRemover(new_array)
def stopword(str_text):
  str_text = stop_words_remover_new.remove(str_text)
  return str_text
df['full_text'] = df['full_text'].apply(lambda x: stopword(x))
print("\nDataFrame setelah Stopwords Removal:")
print(df.head())


DataFrame setelah Stopwords Removal:
                                         created_at  \
conversation_id_str                                   
1858070162362204643  Sun Nov 17 08:50:22 +0000 2024   
1846529554657239219  Wed Oct 16 12:32:07 +0000 2024   
1467691301155979267  Wed Oct 09 13:58:44 +0000 2024   
1836014724523458593  Tue Sep 17 12:09:56 +0000 2024   
1835164951125917967  Sun Sep 15 03:53:14 +0000 2024   

                                                             full_text  
conversation_id_str                                                     
1858070162362204643                lucu banget mahasiswa pencinta alam  
1846529554657239219  rusak mahasiswa pencinta alam jambi menyerang ...  
1467691301155979267  perkumpulan pencinta alam lahir tahun 1953 did...  
1836014724523458593  siaran pers menghadap ukamu ukamu unit pandu l...  
1835164951125917967  siaran pers menghadap ukamu ukamu unit pandu l...  


/tmp/ipython-input-27-1207040705.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_text'] = df['full_text'].apply(lambda x: stopword(x))


In [34]:
# Tokenisasi
tokenized_for_stemming = df['full_text'].apply(lambda x: x.split())
print("\nTokenisasi (sebelum Stemming):")
print(tokenized_for_stemming.head())


Tokenisasi (sebelum Stemming):
conversation_id_str
1858070162362204643            [lucu, banget, mahasiswa, pencinta, alam]
1846529554657239219    [rusak, mahasiswa, pencinta, alam, jambi, meny...
1467691301155979267    [perkumpulan, pencinta, alam, lahir, tahun, 19...
1836014724523458593    [siaran, pers, menghadap, ukamu, ukamu, unit, ...
1835164951125917967    [siaran, pers, menghadap, ukamu, ukamu, unit, ...
Name: full_text, dtype: object


In [ ]:
# Stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
def stemming_text(list_of_tokens): # Mengubah fungsi agar menerima list token
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  stemmed_tokens = [stemmer.stem(w) for w in list_of_tokens]
  return " ".join(stemmed_tokens) # Mengembalikan string
df['full_text'] = tokenized_for_stemming.apply(stemming_text) # Terapkan stemming pada list token
tokenized.to_csv("/content/Stemming.csv", index=False)
print("\nDataFrame setelah Stemming:")
print(df.head())

In [32]:
df = pd.read_csv('/content/preprocessing.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/preprocessing.csv'

In [ ]:
# Labeling
import pandas as pd
classifier = pipeline("sentiment-analysis", model="w11wo/indonesian-roberta-base-sentiment-classifier")

def prediksi_sentimen(teks):
    hasil = classifier(teks)

    return hasil[0]['label']

df['sentimen'] = df['full_text'].apply(prediksi_sentimen)

In [ ]:
df.to_csv('/content/labeling2.csv', index=False)

In [ ]:
sentimen_counts = df.sentimen.value_counts()
sentimen_counts

In [ ]:
df.shape

In [ ]:
# Visualisasi
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_palette("pastel")
plt.figure(figsize=(8, 6))
sns.countplot(x='sentimen', data=df)
plt.title('Analisis Sentimen Publik di X Terhadap Peran Mahasiswa Pencinta Alam dalam Pelestarian Lingkungan')
plt.xlabel('Sentimen')
plt.ylabel('Jumlah')
plt.show()

In [ ]:
data_negatif = df[df['sentimen'] == 'negative']
data_positif = df[df['sentimen'] == 'positive']
data_netral = df[df['sentimen'] == 'neutral']

In [ ]:
all_text_s1 = ' '.join(word for word in data_positif["full_text"])
wordcloud = WordCloud(colormap='Blues', width=1000, height=1000, mode="RGBA", background_color='white').generate(all_text_s1)
plt.figure(figsize=(9, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Visualisasi Kata Positif")
plt.margins(x=0, y=0)
plt.show()

In [ ]:
all_text_s0 = ' '.join(word for word in data_negatif["full_text"])
wordcloud = WordCloud(colormap='Reds', width=1000, height=1000, mode='RGBA', background_color='white').generate(all_text_s0)
plt.figure(figsize=(9, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Visualisasi Kata Negatif")
plt.margins(x=0, y=0)
plt.show()

In [ ]:
all_text_s2 = ' '.join(word for word in data_netral["full_text"])
wordcloud = WordCloud(colormap='Greens', width=1000, height=1000, mode='RGBA', background_color='white').generate(all_text_s2)
plt.figure(figsize=(9, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Visualisasi Kata Netral")
plt.margins(x=0, y=0)
plt.show()

In [ ]:
# Spliting Data
x=df.full_text
y=df.sentimen

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)

In [ ]:
print('Banyak data x_train :',len(x_train))
print('Banyak data x_test  :',len(x_test))
print('Banyak data y_train :',len(y_train))
print('Banyak data y_test  :',len(y_test))

In [ ]:
# Training Data (Naive Bayes)
cvec=CountVectorizer()
tvec=TfidfVectorizer()
hvec=HashingVectorizer()

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf1 = MultinomialNB()

In [ ]:
model1 = Pipeline([('vectorizer',tvec)
                 ,('classifier',clf1)])

In [ ]:
model1.fit(x_train,y_train)

In [ ]:
hasil1=model1.predict(x_test)

In [ ]:
# Confusion Matrix
from sklearn.metrics import ConfusionMatrixDisplay
cm = confusion_matrix(y_test,hasil1)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model1.classes_)
disp.plot(cmap='Blues')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
matrix = classification_report(y_test,hasil1)
print('Classification report : \n',matrix)

In [ ]:
# TESTING
def classify_text(input_text):
    models = [
        ('MultinomialNB', model_multinomialNaiveBayes)
    ]

    results = {}

    for name, model in models:
        prediction = model.predict([input_text])
        results[name] = prediction[0]

    return results

In [ ]:
model_multinomialNaiveBayes = model1.fit(x_train, y_train)

In [ ]:
input_text = input("Masukkan teks yang ingin diklasifikasikan: ")

results = classify_text(input_text)

print("Input kata :", input_text)

print("\nHasil Klasifikasi:")
for model, prediction in results.items():
    print(f"{model}: {prediction}")